# Import libraries

In [ ]:
import os
import numpy as np
import sys
sys.path.append(os.path.abspath('..'))

from src.utils import *
from src import config
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [ ]:
DATA_PATH = os.path.join('../datasets',config.TRAINING_DATAFILE_NAME)

# Model Training

## Data Separation
Separate the data into training data and test data

In [ ]:
label_map = {label:num for num, label in enumerate(config.ACTIONS)}
sequences, labels = [], []
for action in config.ACTIONS:
    for sequence in range(config.NO_SEQUENCES):
        window = []
        for frame_num in range(config.SEQUENCE_LENGTH):
            npy_path = os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num))
            res = np.load(npy_path)
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])
X = np.array(sequences)
y = to_categorical(labels).astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(85, 30, 1662) (85, 3)
(5, 30, 1662) (5, 3)


## Set the log file

In [5]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

## Define the model structure

In [ ]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, activation=config.LSTM_ACTIVATION_FUNCTION, input_shape=(30, 1662), kernel_regularizer=l2(config.L2NORM)))
model.add(LSTM(128, return_sequences=True, activation=config.LSTM_ACTIVATION_FUNCTION, kernel_regularizer=l2(config.L2NORM))) 
model.add(Dropout(config.DROPOUTRATE))
model.add(LSTM(64, return_sequences=False, activation=config.LSTM_ACTIVATION_FUNCTION, kernel_regularizer=l2(config.L2NORM)))
model.add(BatchNormalization())

model.add(Dense(64, activation=config.DENSE_ACTIVATION_FUNCTION, kernel_regularizer=l2(config.L2NORM)))
model.add(Dense(32, activation=config.DENSE_ACTIVATION_FUNCTION, kernel_regularizer=l2(config.L2NORM)))

model.add(Dense(config.ACTIONS.shape[0], activation='softmax'))

## Configure the setting and start the training

Configure the training setting using the setting in config.py

In [ ]:
model.compile(optimizer=config.OPTIMIZER, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Start the training based on those settings

In [ ]:
model.fit(X_train, y_train, epochs=config.EPOCHS, callbacks=[tb_callback])

## Save the model

Set the name of your model and run the code to save the created model as h5 file

In [ ]:
model_name = 'model1'

model.save(os.path.join('..',config.MODEL_SAVE_DIRECTORY , "{}.h5".format(model_name)))


## Test the model

Show all the test data's predicted ansser and the actual answer

In [ ]:
res = model.predict(X_test)
for i in range(len(X_test)):
    print("Predicted: {} , Actual: {}".format(config.ACTIONS[np.argmax(res[i])], config.ACTIONS[np.argmax(y_test[i])]))

Get the confusion matrix and accuracy score of the model

In [ ]:
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)
accuracy_score(ytrue, yhat)